In [2]:
!pip install pathfinding
""" 
for each image in benchmark find path for the given start and end (save images with path) 
for each image save time needed to look for the path
"""
from os import listdir
import time
import cv2 as cv
import math
import os



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:

from pathfinding.core.diagonal_movement import DiagonalMovement
from pathfinding.core.grid import Grid
from pathfinding.finder.a_star import AStarFinder
import numpy as np

In [4]:
#room_path
""" output structure: (room_name,method used, type de test, path, path length, operations, time)"""
class room_path:
    def __init__(self,room_name,method,test,path,path_length,operations,time):
        self.room_name=room_name
        self.method=method
        self.test=test
        self.path=path
        self.path_length=path_length
        self.operations=operations
        self.time=time
    def path_info(self):
        path_info = "room name : " + str(self.room_name)+", test : " + self.test +", path length : " + str(self.path_length) + ", time : " + str(self.time) 
        return path_info
        



In [5]:
#utils
def image_none(img_path):
    image=cv.imread(img_path)
    return image

def show_image(image):
    cv.imshow('image', image) 
    cv.waitKey(0)
    cv.destroyAllWindows()

def euclidean_distance(node1, node2):
    return math.sqrt((node1[0] - node2[0])**2 + (node1[1] - node2[1])**2)

def manhattan_distance(node1, node2):
    return abs(node1[0] - node2[0]) + abs(node1[1] - node2[1])
  

def octile_distance(node1,node2):
    dx= node2[0] - node1[0]
    dy= node2[1] - node1[1]
    f = math.sqrt(2) - 1
    if dx < dy:
        return f * dx + dy
    else:
        return f * dy + dx


def octile(path):
    distance=0
    for i in range(len(path)-1):
        distance+=octile_distance(path[i],path[i+1])
    return distance


def coloring_pixel(image,color,pixel):
    if image is  not None:
        image[pixel[0],pixel[1]]=color
        return image

def define_start_goal(image,start,goal):
    start_color = [255, 0, 102] #pink 
    goal_color =  [0, 0, 204] #blue
    if image is not None:
        # should not be obstacle 
        if np.array_equal(image[start[0],start[1]],[229, 229, 229]) and np.array_equal(image[goal[0],goal[1]],[229, 229, 229]):
            image = coloring_pixel(image,start_color,start)
            image = coloring_pixel(image,goal_color,goal) 
            return image
        else: 
            print("Goal or Start is an obstacle, Redefine it!")
            return image

def draw_path(img_path,path):    
    image=cv.imread(img_path)
    path = set(path)
    if image is not None:
        for tuple in path:
            image[tuple[1],tuple[0]]=[0, 0, 255] #red
    return image

def save_image(img_path,path,image_name):
    image=cv.imread(img_path)
    new_image = draw_path(image,path)
    saving_path = '/home/aisha/PFE/implementations/UAV-Path-Planning/src/tests/output/'
    cv.imwrite(os.path.join(saving_path , image_name), new_image)    

def isValid(img_path,node):
    image=cv.imread(img_path,0)
    if image is not None:
        return  image[node[0],node[1]] == 229 



def imageToMatrix(img_path):
    image=cv.imread(img_path,0)
    rows,cols=(1073,1073)    
    # image (grey scale) to a matrix of 0 and 1 
    for i in range(rows):
        for j in range(cols):
            if image[i,j] == 0 or image[i,j] == 99: 
                image[i,j] = 0 #obstacle
            elif image[i,j] == 229:
                image[i,j] = 1
    return image                
    


In [6]:
#Astar

from numba import jit, cuda

# function optimized to run on gpu 
@jit(target_backend='cuda')
def Astar(img_path,start,end,image_name,test):
    image= imageToMatrix(img_path)
    grid = Grid(matrix=image)
    start = grid.node(start[0],start[1])
    end = grid.node(end[0],end[1])
    finder = AStarFinder(diagonal_movement=DiagonalMovement.always)
    path, runs = finder.find_path(start, end, grid)
    print("path length: ",octile(path) , "runs: " , runs)
        
    #Draw path in the image and save it
    image=cv.imread(img_path)
    new_image = draw_path(img_path,path)
    saving_path = '/content/'+test
    cv.imwrite(os.path.join(saving_path ,image_name), new_image)
    
    return path, runs , octile(path)



In [ ]:

folder_dir = "/content/"
file= open("/content/colab_results.txt","w")
#-------------------EASY TEST---------------------------------------- 

file.write("\nEASY TEST \n")

start = (29,38)
end = (21,50)

for image in listdir(folder_dir):
    img_path=folder_dir+str(image)
    print(img_path)
    img=cv.imread(img_path,0)
    if img is not None:
        if isValid(img_path,start):
            if isValid(img_path,end):
                #print(str(image))
                start_time = time.time()
                path, runs , length = Astar(img_path,start,end,"new_"+str(image),"easy")
                #print("--- %s seconds ---" % (time.time() - start_time))
                pathInfo=(room_path("new_"+str(image),"astar","easy" , path,length,runs,time.time() - start_time)).path_info()
                print(pathInfo)
                file.write(pathInfo+"\n")
                
            else:
                print(str(image) ,"invalid end")
        else:
            print(str(image),"invalid start")
    else:
        print("image is None")
    
    
#-------------------Medium TEST---------------------------------------- 
file.write("\nMEDIUM TEST \n")

# CHANGE START , END

start = (18, 30)
end = (21, 1001)


for image in listdir(folder_dir):
    img_path=folder_dir+"/"+str(image)
    img=cv.imread(img_path,0)
    if img is not None:
        if isValid(img_path,start):
            if isValid(img_path,end):
                #print(str(image))
                start_time = time.time()
                path, runs , length = Astar(img_path,start,end,"new_"+str(image),"medium")
                #print("--- %s seconds ---" % (time.time() - start_time))
                pathInfo=(room_path("new_"+str(image),"astar","medium" , path,length,runs,time.time() - start_time)).path_info()
                print(pathInfo)
                file.write(pathInfo+"/n")
            else:
                print(str(image) ,"invalid end")
        else:
            print(str(image),"invalid start")
    else:
        print("image is None")


#-------------------HARD TEST---------------------------------------- 

file.write("\nHARD TEST \n")

# CHANGE START , END

start = (12, 30)
end = (1040, 990)


for image in listdir(folder_dir):
    img_path=folder_dir+"/"+str(image)
    img=cv.imread(img_path,0)
    if img is not None:
        if isValid(img_path,start):
            if isValid(img_path,end):
                #print(str(image))
                start_time = time.time()
                path, runs , length = Astar(img_path,start,end,"new_"+str(image),"hard")
                #print("--- %s seconds ---" % (time.time() - start_time))
                pathInfo = (room_path("new_"+str(image),"astar","hard" ,path, length,runs,time.time() - start_time)).path_info()
                print(pathInfo)
                file.write(pathInfo+"/n")
            else:
                print(str(image) ,"invalid end")
        else:
            print(str(image),"invalid start")
    else:
        print("image is None")
    




/content/.config
image is None
/content/8room_004.png


<ipython-input-6-2ffa90b9f2c4>:6: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "Astar" failed type inference due to: Untyped global name 'imageToMatrix': Cannot determine Numba type of <class 'function'>

File "<ipython-input-6-2ffa90b9f2c4>", line 8:
def Astar(img_path,start,end,image_name,test):
    image= imageToMatrix(img_path)
    ^

  @jit(target_backend='cuda')
/usr/local/lib/python3.7/dist-packages/numba/core/object_mode_passes.py:152: NumbaWarning: Function "Astar" was compiled in object mode without forceobj=True.

File "<ipython-input-6-2ffa90b9f2c4>", line 7:
@jit(target_backend='cuda')
def Astar(img_path,start,end,image_name,test):
^

  state.func_ir.loc))
/usr/local/lib/python3.7/dist-packages/numba/core/object_mode_passes.py:162: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https:

path length:  8.686291501015239 runs:  39
room name : new_8room_004.png, test : easy, path length : 8.686291501015239, time : 7.616967678070068
/content/16room_004.png
path length:  8.686291501015239 runs:  39
room name : new_16room_004.png, test : easy, path length : 8.686291501015239, time : 7.273627758026123
/content/32room_008.png
path length:  8.686291501015239 runs:  36
room name : new_32room_008.png, test : easy, path length : 8.686291501015239, time : 7.268943786621094
/content/16room_009.png
path length:  8.686291501015239 runs:  35
room name : new_16room_009.png, test : easy, path length : 8.686291501015239, time : 7.293569326400757
/content/64room_009.png
path length:  8.686291501015239 runs:  36
room name : new_64room_009.png, test : easy, path length : 8.686291501015239, time : 7.575352191925049
/content/8room_006.png
path length:  8.686291501015239 runs:  39
room name : new_8room_006.png, test : easy, path length : 8.686291501015239, time : 6.814105033874512
/content/16ro